# Milestone 2

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

## Explorting dataset:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_DIR = "/content/drive/MyDrive/tfds_data"

trivia_train_subset, info = tfds.load(
    "trivia_qa",
    split="train[:15000]",
    with_info=True,
    data_dir=DATA_DIR,
    download=True   # only downloads if it’s not already in DATA_DIR
)

trivia_validation = tfds.load(
    "trivia_qa",
    split="validation",
    data_dir=DATA_DIR
)


In [ ]:

for sample in trivia_train_subset.take(1):
    print(sample)

{'answer': {'aliases': <tf.Tensor: shape=(33,), dtype=string, numpy=
array([b'La Republique francaise', b'Franciaorsz\xc3\xa1g',
       b'La Republique fran\xc3\xa7aise', b'Belle France', b'FRANCE',
       b'Republique fran\xc3\xa7aise', b'France', b'Ranska',
       b'Franciaorszag', b'French Republic',
       b'R\xc3\xa9publique francaise',
       b'R\xc3\xa9publique fran\xc3\xa7aise', b'ISO 3166-1:FR',
       b'Frankreich', b'France (country)', b'Etymology of France',
       b'FrancE', b'La R\xc3\xa9publique fran\xc3\xa7aise',
       b'People of France', b'Republic of France', b'Fracne',
       b'Le France', b'The French Republic', b'Republique francaise',
       b'The Republic of France', b'La R\xc3\xa9publique francaise',
       b'La France', b'Rain in France', b'Republic of france', b'FRance',
       b'Frakkland', b'Climate of France', b'Franse'], dtype=object)>, 'matched_wiki_entity_name': <tf.Tensor: shape=(), dtype=string, numpy=b''>, 'normalized_aliases': <tf.Tensor: shape=(27

In [ ]:
info.features

FeaturesDict({
    'answer': FeaturesDict({
        'aliases': Sequence(Text(shape=(), dtype=string)),
        'matched_wiki_entity_name': Text(shape=(), dtype=string),
        'normalized_aliases': Sequence(Text(shape=(), dtype=string)),
        'normalized_matched_wiki_entity_name': Text(shape=(), dtype=string),
        'normalized_value': Text(shape=(), dtype=string),
        'type': Text(shape=(), dtype=string),
        'value': Text(shape=(), dtype=string),
    }),
    'entity_pages': Sequence({
        'doc_source': Text(shape=(), dtype=string),
        'filename': Text(shape=(), dtype=string),
        'title': Text(shape=(), dtype=string),
        'wiki_context': Text(shape=(), dtype=string),
    }),
    'question': Text(shape=(), dtype=string),
    'question_id': Text(shape=(), dtype=string),
    'question_source': Text(shape=(), dtype=string),
    'search_results': Sequence({
        'description': Text(shape=(), dtype=string),
        'filename': Text(shape=(), dtype=string),

##Data Cleaning

In [ ]:
def select_features(example):
    return {
        "question_id": example["question_id"],
        "question":    example["question"],
        "context":     example["search_results"]["search_context"][0],
        "more_context": example["entity_pages"]["wiki_context"][0],
        "answer":      example["answer"]["value"],
    }

ds_cleaned = trivia_train_subset.map(
    select_features,
    num_parallel_calls=tf.data.AUTOTUNE
)

# Print the very first cleaned example
for ex in ds_cleaned.take(1):
    print(ex)


{'question_id': <tf.Tensor: shape=(), dtype=string, numpy=b'sfq_9115'>, 'question': <tf.Tensor: shape=(), dtype=string, numpy=b'Modelled on the Spanish bullfight, in which country did the Paso Doble dance originate?'>, 'context': <tf.Tensor: shape=(), dtype=string, numpy=b"Latin American Guide: History Of The Paso Doble Dance\nHistory Of The Paso Doble Dance\nBy Melanie LaPatin\nWho says the French can't be Spanish? Witness the Paso Doble. Meaning double step in Spanish, this ballroom dance for couples is set to Spanish march music that is reminiscent of a bullfight in the heart of Spain.\nIt is the same style of music that is played when a bullfighter enters the ring to salute the audience before the fight, and again at the end of the fight, just before the kill. This assumes, of course, that the bull doesnt win the fight.\nThe dance is lively and dramatic, with long sweeping steps and aggressive movements. It is a dance of preparation for battle and triumph in combat, just like the b